In [3]:
import os
import cv2
import json
import numpy as np 
from PIL import Image

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = 'data/train/'

In [ ]:
# Masks generation, creating file structure for images and masks

source_folder = "images"
json_path = "crosswalks.json"              
file_polygons = {}                                   

with open(json_path) as f:
    data = json.load(f)

def add_to_dict(data, itr, key, count):
    try:
        x_points = data[itr]["regions"][count]["shape_attributes"]["all_points_x"]
        y_points = data[itr]["regions"][count]["shape_attributes"]["all_points_y"]
    except:
        return
    
    all_points = []
    for i, x in enumerate(x_points):
        all_points.append([x, y_points[i]])
    
    file_polygons[key] = all_points

for itr in data:
    file_name_json = data[itr]["filename"]
    sub_count = 0
    
    if len(data[itr]["regions"]) > 1:
        for _ in range(len(data[itr]["regions"])):
            key = file_name_json[:-4] + "*" + str(sub_count+1)
            add_to_dict(data, itr, key, sub_count)
            sub_count += 1
    else:
        add_to_dict(data, itr, file_name_json[:-4], sub_count)

for file_name in os.listdir(source_folder):
    to_save_folder = os.path.join(source_folder, file_name[:-4])
    image_folder = os.path.join(to_save_folder, "images")
    mask_folder = os.path.join(to_save_folder, "masks")
    curr_img = os.path.join(source_folder, file_name)
    
    try:
        os.mkdir(to_save_folder)
        os.mkdir(image_folder)
        os.mkdir(mask_folder)
        os.rename(curr_img, os.path.join(image_folder, file_name))
    except:
        pass
    
for polygone in file_polygons:
    num_masks = polygone.split("*")
    to_save_folder = os.path.join(source_folder, num_masks[0])
    mask_folder = os.path.join(to_save_folder, "masks")
    
    im = cv2.imread(os.path.join(to_save_folder, "images", num_masks[0] + ".png"))   
    mask = np.zeros((im.shape[0], im.shape[1]))
    arr = np.array(file_polygons[polygone])
    
    cv2.fillPoly(mask, [arr], color=(255))
    
    if len(num_masks) > 1:
        cv2.imwrite(os.path.join(mask_folder, polygone.replace("*", "_") + ".png"), mask)    
    else:
        cv2.imwrite(os.path.join(mask_folder, polygone + ".png"), mask)

In [6]:
# Get data from file structure

train_ids = next(os.walk(TRAIN_PATH))[1]

X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

print("X_train",X_train.shape)
print("Y_train",Y_train.shape)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    if id_[0] == '.':
        continue
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        if mask_file[0] == '.':
            continue
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask
print('Done!')

StopIteration: 